In [ ]:
import msprime

print('*'*50)
print("DEPRECATED 06 AUG 2025")
print("Use scripts/")
print('*'*50)

n_individuals = 100000    
sequence_length = 500000   
mutation_rate = 1e-8         
recombination_rate = 1e-7
random_seed = 42

ts = msprime.sim_ancestry(
    samples=n_individuals,     
    sequence_length=sequence_length,
    recombination_rate=recombination_rate,
    population_size=10_000,  
    ploidy=2,
    random_seed=random_seed
)

ts = msprime.sim_mutations(
    ts,
    rate=mutation_rate,
    random_seed=random_seed + 1
)

print(f"Simulated {ts.num_individuals} individuals")
print(f"Simulated {ts.num_sites} variants (segregating sites)")

individual_names = [f"indiv{i+1}" for i in range(ts.num_individuals)]

output_dir = f"{n_individuals:.2e}_{ts.num_sites}"

# if output_dir does not exist, create it
import os
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

with open(f"{output_dir}/output_{output_dir}.vcf", "w") as f:
    ts.write_vcf(f, individual_names=individual_names)

print(f"VCF written to {output_dir}/output_{output_dir}.vcf")

Simulated 100000 individuals
Simulated 2578 variants (segregating sites)
VCF written to 1.00e+05_2578/output_1.00e+05_2578.vcf


In [26]:

G = ts.genotype_matrix() 
num_sites, num_haplotypes = G.shape
n_individuals = num_haplotypes // 2



In [ ]:
import numpy as np

# allele_counts = G.sum(axis=1)
# maf = np.minimum(allele_counts, num_haplotypes - allele_counts) / num_haplotypes



In [28]:
# import allel
# import numpy as np
# import multiprocessing as mp

# G_alt = G.T.reshape((n_individuals, num_sites, 2))
# gn = allel.GenotypeArray(G_alt)
# gn_alt = gn.to_n_alt()  # shape: (n_individuals, num_sites)

# def compute_ld_scores(index_range):
#     partial_scores = np.zeros(len(index_range))
#     for idx, i in enumerate(index_range):
#         start = max(0, i - window_size)
#         end = min(num_sites, i + window_size + 1)
#         if end - start < 2:
#             partial_scores[idx] = 1.0
#             continue
#         r = allel.rogers_huff_r(gn_alt[:, start:end])
#         r2 = r ** 2
#         partial_scores[idx] = r2[i - start].sum()
#     return index_range, partial_scores

# window_size = 100
# num_sites = gn_alt.shape[1]
# n_processes = mp.cpu_count()
# print(n_processes)
# chunk_size = num_sites // n_processes

# index_chunks = [list(range(i, min(i + chunk_size, num_sites))) for i in range(0, num_sites, chunk_size)]

# with mp.Pool(processes=n_processes) as pool:
#     results = pool.map(compute_ld_scores, index_chunks)

# ld_scores = np.zeros(num_sites)
# for idxs, scores in results:
#     ld_scores[idxs] = scores


In [29]:
# with open("maf.ld.txt", "w") as f:
#     f.write("MAF\tldscore\n")
#     for m, ld in zip(maf, ld_scores):
#         f.write(f"{m:.6f}\t{ld:.5f}\n")

# print(f"Wrote MAF and LD scores for {num_sites} variants to maf.ld.txt")

In [33]:
# 20% annotation
annotations = np.random.binomial(1, 0.2, size=num_sites)

np.savetxt(f"{output_dir}/annotations.txt", annotations, fmt="%d")
print(f"Wrote binary annotations to {output_dir}/annotations.txt")


Wrote binary annotations to 1.00e+05_2578/annotations.txt


In [34]:
E = np.random.normal(0, 1, size=(n_individuals, 1))
np.savetxt(f"{output_dir}/env.txt", E, fmt="%.6f")
print(f"Wrote environmental data for {n_individuals} individuals to {output_dir}/env.txt")

Wrote environmental data for 100000 individuals to 1.00e+05_2578/env.txt


In [32]:
# %%bash

# slow fib busy work
# l=35
# u=40
# echo $l ".." $u
# time for n in $(seq $l $u); do
#   python3 -c 'f=lambda n:(n>1 and f(n-1)+f(n-2))or n;print(__import__("sys").argv[1], f(int(__import__("sys").argv[1])))' "$n"
# done